In [45]:
import pandas as pd
import requests
import os
import re
import numpy as np


In [35]:
def limpiar_pgn(pgn_raw):
    """
    Recibe un PGN crudo (con \n, relojes, etc.) y devuelve un PGN limpio y legible.
    """
    # 1. Reemplazar secuencias \n por saltos de línea reales
    pgn = pgn_raw.replace("\\n", "\n")

    # 2. Quitar anotaciones de reloj {[%clk ...]}
    import re
    pgn = re.sub(r"\{.*?\}", "", pgn)

    # 3. Quitar espacios dobles y limpiar
    pgn = re.sub(r"\s+", " ", pgn)
    pgn = pgn.replace(" ]", "]")  # arreglar corchetes
    
    # 4. Reconvertir a saltos de línea normales en headers
    pgn = pgn.replace("] [", "]\n[")

    # 5. Asegurar doble salto antes de jugadas
    if "1." in pgn:
        pgn = pgn.replace("] 1.", "]\n\n1.")

    return pgn.strip()

In [3]:
output_dir = os.path.join("..", "data")

players = pd.read_csv(os.path.join(output_dir, "players_stats.csv"))
games = pd.read_csv(os.path.join(output_dir, "chess_games.csv"))


In [38]:
games["pgn"] = games["pgn"].apply(limpiar_pgn)


In [42]:
output_path = os.path.join(output_dir, "chess_games.csv")
games.to_csv(output_path, index=False, encoding="utf-8")

In [43]:
hikaru_games = games[(games["white_username"] == player)|(games["black_username"] == player)]

In [49]:
hikaru_games["rival_elo"] = np.where(
    hikaru_games["black_username"] == player,
    hikaru_games["white_rating"],
    hikaru_games["black_rating"]
)

hikaru_games["player_score"] = np.where(
    hikaru_games["white_username"] == player,
    np.where(hikaru_games["white_result"] == "win", 1, 0), 
    np.where(hikaru_games["black_result"] == "win", 1, 0)   
)

C:\Users\enriq\AppData\Local\Temp\ipykernel_33500\1059263251.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hikaru_games["rival_elo"] = np.where(
C:\Users\enriq\AppData\Local\Temp\ipykernel_33500\1059263251.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hikaru_games["player_score"] = np.where(


In [ ]:
hikaru_games.sort_values("rival_elo", ascending=False)[hikaru_games["player_score"]== 1].head(5)

C:\Users\enriq\AppData\Local\Temp\ipykernel_33500\156113450.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  hikaru_games.sort_values("rival_elo", ascending=False)[hikaru_games["player_score"]== 1].head(5)


,url,pgn,time_control,time_class,accuracies_white,accuracies_black,white_username,white_rating,white_result,black_username,black_rating,black_result,eco,eco_name,rules,score_white,score_black,rival_elo,player_score
823,https://www.chess.com/game/live/146322210861,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+1,blitz,92.21,84.66,Hikaru,3412,win,HansOnTwitch,3256,resigned,A45,Amazon Attack: Siberian Attack,chess,1.0,1.0,3256,1
816,https://www.chess.com/game/live/146307802621,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+1,blitz,78.15,80.30,vi_pranav,3246,resigned,Hikaru,3400,win,C28,"Bishop's Opening: Vienna Hybrid, Hromádka Vari...",chess,0.0,0.0,3246,1
1105,https://www.chess.com/game/live/147916998505,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,85.03,90.48,DanielNaroditsky,3219,resigned,Hikaru,3292,win,C46,Three Knights Opening,chess,0.0,0.0,3219,1
815,https://www.chess.com/game/live/146307200289,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180+1,blitz,78.48,78.91,Hikaru,3395,win,DenLaz,3213,timeout,E73,King's Indian Defense: Averbakh Variation,chess,1.0,1.0,3213,1
1106,https://www.chess.com/game/live/147917004319,"[Event ""Live Chess""]\n[Site ""Chess.com""]\n[Dat...",180,blitz,87.55,77.25,Hikaru,3298,win,DanielNaroditsky,3213,resigned,B22,Sicilian Defense: Alapin Variation,chess,1.0,1.0,3213,1
